In [56]:
import pandas as pd
import numpy as np

In [ ]:
FAU = ["AU01", "AU02", "AU04", "AU05", "AU06", "AU07", "AU09", "AU10", "AU11", "AU12", "AU13", "AU14", "AU15", "AU16", "AU17", "AU18", "AU20", "AU22", "AU23", "AU24", "AU25", "AU28", "AU41", "AU42", "AU43", "AU44", "AU45", "AU46"]

In [3]:
df = pd.read_csv("./OpenFace/output/russell_test.csv")
df.head()

,frame,face_id,timestamp,confidence,success,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,1,0,0.000,0.98,1,0.0,0.0,0.11,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0,0.033,0.98,1,0.0,0.0,0.38,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0,0.067,0.98,1,0.0,0.0,0.42,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0,0.100,0.98,1,0.0,0.0,0.23,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0,0.133,0.98,1,0.0,0.0,0.07,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
df_fau = pd.DataFrame(df.iloc[:, 5:])
df_fau.columns = [c.strip() for c in df_fau.columns]
df_fau

,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,AU12_r,AU14_r,...,AU12_c,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c
0,0.00,0.0,0.11,0.00,0.00,0.00,0.00,0.00,0.40,0.00,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.00,0.0,0.38,0.00,0.00,0.00,0.00,0.00,0.41,0.16,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.00,0.0,0.42,0.00,0.00,0.07,0.00,0.00,0.40,0.16,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00,0.0,0.23,0.00,0.00,0.07,0.00,0.00,0.43,0.16,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.00,0.0,0.07,0.00,0.00,0.07,0.00,0.00,0.49,0.07,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,1.92,0.0,0.00,0.27,0.75,0.00,0.87,0.70,1.04,2.02,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
140,1.35,0.0,0.00,0.15,0.47,0.00,0.61,0.53,0.70,1.47,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
141,0.82,0.0,0.00,0.06,0.33,0.00,0.97,0.33,0.88,1.53,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
142,0.13,0.0,0.00,0.06,0.34,0.00,1.36,0.00,0.95,1.49,...,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [61]:
cond1 = df_fau[" AU01_c"] == 1
cond2 = df_fau[" AU01_r"] >= 0.5
df_fau["AU01"] = np.where(cond1 & cond2, 1, 0)
df_fau["AU01"]

0      0
1      0
2      0
3      0
4      0
      ..
139    0
140    0
141    0
142    0
143    0
Name: AU01, Length: 144, dtype: int32